In [1]:
import torch
from torch import nn
from torch.nn import init

In [2]:
net=nn.Sequential(nn.Linear(4,3),nn.ReLU(),nn.Linear(3,1))

print(net)

X=torch.rand(2,4)
Y=net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


# 访问模型参数

In [3]:
print(type(net.named_parameters()))
for name,param in net.named_parameters():
    print(name,param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [4]:
for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [5]:
class MyModel(nn.Module):
    def __init__(self,**kwargs):
        super(MyModel,self).__init__(**kwargs)
        self.weight1=nn.Parameter(torch.rand(20,20))
        self.weight2=torch.rand(20,20)
    def forward(self,x):
        pass

n=MyModel()
for name,param in n.named_parameters():
    print(name)

weight1


In [6]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) # 反向传播前梯度为None
Y.backward()
print(weight_0.grad)

tensor([[ 0.3658,  0.0909,  0.0494,  0.1454],
        [ 0.4919, -0.2522, -0.4944,  0.2880],
        [ 0.1323,  0.4627, -0.2840,  0.0963]])
None
tensor([[-0.0833, -0.1618, -0.1965, -0.1018],
        [ 0.0328,  0.0498,  0.0644,  0.0629],
        [-0.2936, -0.5703, -0.6928, -0.3589]])


# 初始化模型参数

In [7]:
for name,param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param,mean=0,std=0.01)
        print(name,param.data)

0.weight tensor([[-0.0094,  0.0019,  0.0099,  0.0136],
        [ 0.0069,  0.0001,  0.0165,  0.0103],
        [-0.0055,  0.0089,  0.0153,  0.0040]])
2.weight tensor([[ 0.0208,  0.0044, -0.0007]])


In [8]:
#下面使用常数来初始化权重参数。
for name,param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val=0)
        print(name,param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


# 自定义初始化方法

In [9]:
def normal_(tensor,mean=0,std=1):
    with torch.no_grad():
        return tensor.normal_(mean,std)

In [10]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)
        tensor*=(tensor.abs()>=5).float()

for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name,param.data)

0.weight tensor([[-0.0000, -8.1454, -0.0000,  0.0000],
        [ 0.0000, -0.0000,  6.5958,  6.0654],
        [-0.0000,  6.6375,  6.4642,  6.8702]])
2.weight tensor([[ 9.3164, -5.3036,  6.0350]])


In [11]:
#此外，参考2.3.2节，我们还可以通过改变这些参数的data来改写模型参数值同时不会影响梯度:
for name,param in net.named_parameters():
    if 'bias' in name:
        param.data+=1
        print(name,param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


# 共享模型参数

In [12]:
linear=nn.Linear(1,1,bias=False)
net=nn.Sequential(linear,linear)
print(net)
for name,param in net.named_parameters():
    init.constant_(param,val=3)
    print(name,param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [13]:
#在内存中，这两个线性层其实一个对象:
print(id(net[0])==id(net[1]))
print(id(net[0].weight)==id(net[1].weight))

True
True


In [14]:
#因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:
x=torch.ones(1,1)
y=net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
